In [15]:
import cv2
import numpy as np
import pandas as pd
import os
import shutil

In [16]:
def average_BGR(image):
    avg_blue = image[:,:,0].mean()
    avg_green = image[:,:,1].mean()
    avg_red = image[:,:,2].mean()

    return [avg_blue,avg_green,avg_red]

def average_HSV(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    avg_hue = image[:,:,0].mean()
    avg_saturation = image[:,:,1].mean()
    avg_value = image[:,:,2].mean()

    return avg_hue, avg_saturation, avg_value

In [17]:
list_avg_blue = []
list_avg_green = []
list_avg_red = []

list_avg_hue=[]
list_avg_saturation=[]
list_avg_value=[]
X1 = []
X2 = []
list_image = []
list_image_path=[]
for dirname, _, filenames in os.walk('D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix'):
    for filename in filenames:
        image_path = os.path.join(dirname, filename)
        list_image_path.append(image_path)
        image = cv2.imread(image_path)
        list_image.append(image)
        avg_BGR = average_BGR(image)
        X1.append(avg_BGR)
        list_avg_blue.append(avg_BGR[0])
        list_avg_green.append(avg_BGR[1])
        list_avg_red.append(avg_BGR[2])
        avg_HSV= average_HSV(image)
        list_avg_hue.append(avg_HSV[0])
        list_avg_saturation.append(avg_HSV[1])
        list_avg_value.append(avg_HSV[2])
        X2.append(avg_HSV)

In [18]:
from sklearn.cluster import KMeans

cluster = 11

kmeans = KMeans(n_clusters=cluster,random_state =0)

X2 = np.array(X2)

kmeans.fit(X2)


print('Centers found by scikit-learn:')
print(kmeans.cluster_centers_)

pred_label = kmeans.predict(X2)


output_dir = 'D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1'

# Kiểm tra xem thư mục tồn tại chưa, nếu không, tạo mới
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Tạo đường dẫn đến file
output_file = os.path.join(output_dir, 'cluster_labels.txt')


# Mở file để ghi dữ liệu
with open(output_file, 'w') as file:
    # Ghi nhãn của từng điểm dữ liệu
    np.savetxt(file, kmeans.labels_, fmt='%d')


output_file = os.path.join(output_dir, 'cluster_center.txt')

with open(output_file, 'w') as file:
    # Ghi trung tâm của các cụm

    np.savetxt(file, kmeans.cluster_centers_)


dict = {}

for i in range(len(list_image_path)):
    dict[list_image_path[i]] = pred_label[i]

for i in range(cluster):
    folder_name = str(i)
    os.mkdir(os.path.join('D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1', folder_name))

for key, value in dict.items():
    for dirname, child_folders, filenames in os.walk('kmeans1'):
        for folder in child_folders:
            if str(value) == folder:
                shutil.copy(key, os.path.join(dirname, folder))


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Centers found by scikit-learn:
[[ 74.22933333  15.32813333 165.43575333]
 [ 22.57452     92.76961333 173.80278667]
 [ 94.85196     94.31786    107.31125333]
 [ 17.480295    53.495255   222.478025  ]
 [ 39.788485   127.585285   100.630485  ]
 [ 46.77166562  55.72717812 155.56149063]
 [ 20.26515333 172.23953333 173.90971333]
 [ 24.29108462 105.83596923 200.83433846]
 [ 27.44935714  26.31848571 179.77884286]
 [ 21.954924    69.96274    197.133108  ]
 [ 22.03905926  69.8302037  144.14539259]]


In [19]:
def kmeans_2(folder_path, mod):
    images = []
    list_image_path = []
    for filename in os.listdir(folder_path):
        # print(folder_path)
        image_path = os.path.join(folder_path, filename)
        # print(image_path)
        list_image_path.append(image_path)
        image = cv2.imread(image_path)
        # print(image)
        avg_HSV= average_HSV(image)
        images.append(avg_HSV)
    
    kmeans_second = KMeans(n_clusters=mod,random_state =0)
    kmeans_second.fit(np.array(images))
    label = kmeans_second.predict(images)
    
    dict = {}
    for i in range((len(list_image_path))):
        dict[list_image_path[i]] = label[i]
    
    for key, value in dict.items():
        print(key, ':', value)

    for i in range(mod):
        folder_name = str(i)
        os.mkdir(os.path.join(folder_path, folder_name))

    for key, value in dict.items():
        for dirname, child_folders, _ in os.walk(folder_path):
            for folder in child_folders:
                if str(value) == folder:
                    shutil.move(key, os.path.join(dirname, folder))

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Tạo đường dẫn đến file
    output_file = os.path.join(folder_path, 'cluster_labels.txt')


    # Mở file để ghi dữ liệu
    with open(output_file, 'w') as file:
        # Ghi nhãn của từng điểm dữ liệu
        np.savetxt(file, kmeans_second.labels_, fmt='%d')


    output_file = os.path.join(folder_path, 'cluster_center.txt')

    with open(output_file, 'w') as file:
        # Ghi trung tâm của các cụm

        np.savetxt(file, kmeans_second.cluster_centers_)    

folder = 'D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1'

dict_km2 = {}

for i in range(cluster):
    folder_path = os.path.join(folder, str(i))
    mod = len(os.listdir(folder_path)) // 13
    if mod > 1:
        kmeans_2(folder_path, mod)
        

# print(result)

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set

D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Banana (14).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (01).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (02).jpg : 0
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (03).jpg : 0
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (04).jpg : 0
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (05).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (06).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (07).jpg : 0
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (08).jpg : 0
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (09).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (10).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (11).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (12).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1\1\Kiwi (13).jpg : 1
D:\P